In [17]:
import os
import tarfile
import urllib
import re
import bz2
import pandas as pd
import numpy as np

In [2]:
DOWNLOAD_ROOT = "https://spamassassin.apache.org"
SPAM_PATH = os.path.join("datasets", "spam")
SPAM_URL = DOWNLOAD_ROOT + "/old/publiccorpus/"

def fetch_spam_data(spam_url=SPAM_URL, spam_path=SPAM_PATH):
    os.makedirs(spam_path, exist_ok=True)
    urlpath = urllib.request.urlopen('https://spamassassin.apache.org/old/publiccorpus/')
    string = urlpath.read().decode('utf-8')
    pattern = re.compile('href=".*\.bz2"')
    filelist = [f[6:-1] for f in pattern.findall(string)]
    for f in filelist:
        bz2path = os.path.join(spam_path, f)
        urllib.request.urlretrieve(spam_url+f, bz2path)
        with bz2.BZ2File(bz2path) as bz2_file:
            data = bz2_file.read()
        tarpath = bz2path[:-4]
        open(tarpath, 'wb').write(data)
        with tarfile.open(tarpath) as tar_file:
            tar_file.extractall(path=spam_path)
        os.remove(bz2path)
        os.remove(tarpath)

In [3]:
# fetch_spam_data()

In [4]:
contents = []
cats = []

for cat in os.listdir(SPAM_PATH):
    cat_path = os.path.join(SPAM_PATH, cat)
    cat_files = os.listdir(cat_path)
    cat_contents = [open(os.path.join(cat_path, f), encoding='ansi').read() for f in cat_files if f != "cmds"]
    contents += cat_contents
    cats += [cat] * len(cat_contents)
    
emails = pd.DataFrame(data={'Content': contents, 'Cat': cats})

In [5]:
emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9349 entries, 0 to 9348
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Content  9349 non-null   object
 1   Cat      9349 non-null   object
dtypes: object(2)
memory usage: 146.2+ KB


In [6]:
emails['Cat'].value_counts()

easy_ham      5051
easy_ham_2    1400
spam_2        1397
spam          1001
hard_ham       500
Name: Cat, dtype: int64

In [7]:
emails["Cat"] = emails.apply(lambda row: row["Cat"][:-2] if (row["Cat"][-2] == '_' and row["Cat"][-1].isdigit()) else row["Cat"], axis=1)

In [8]:
emails['Cat'].value_counts()/len(emails)

easy_ham    0.690020
spam        0.256498
hard_ham    0.053482
Name: Cat, dtype: float64

In [9]:
emails['Label'] = emails['Cat'].str.contains('spam').astype(int)

In [10]:
emails

,Content,Cat,Label
0,From exmh-workers-admin@redhat.com Thu Aug 22...,easy_ham,0
1,From Steve_Burt@cursor-system.com Thu Aug 22 ...,easy_ham,0
2,From timc@2ubh.com Thu Aug 22 13:52:59 2002\n...,easy_ham,0
3,From irregulars-admin@tb.tf Thu Aug 22 14:23:...,easy_ham,0
4,From Stewart.Smith@ee.ed.ac.uk Thu Aug 22 14:...,easy_ham,0
...,...,...,...
9344,From Professional_Career_Development_Institute...,spam,1
9345,From tba@insiq.us Wed Dec 4 11:46:34 2002\nR...,spam,1
9346,Return-Path: <raye@yahoo.lv>\nReceived: from u...,spam,1
9347,From cweqx@dialix.oz.au Tue Aug 6 11:03:54 2...,spam,1


In [11]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(emails, emails["Cat"]):
    strat_train_set = emails.loc[train_index]
    strat_test_set = emails.loc[test_index]

In [12]:
strat_train_set['Cat'].value_counts()/len(strat_train_set)

easy_ham    0.690066
spam        0.256451
hard_ham    0.053483
Name: Cat, dtype: float64

In [13]:
strat_test_set['Cat'].value_counts()/len(strat_test_set)

easy_ham    0.689840
spam        0.256684
hard_ham    0.053476
Name: Cat, dtype: float64

In [29]:
emails

,Content,Cat,Label
0,From exmh-workers-admin@redhat.com Thu Aug 22...,easy_ham,0
1,From Steve_Burt@cursor-system.com Thu Aug 22 ...,easy_ham,0
2,From timc@2ubh.com Thu Aug 22 13:52:59 2002\n...,easy_ham,0
3,From irregulars-admin@tb.tf Thu Aug 22 14:23:...,easy_ham,0
4,From Stewart.Smith@ee.ed.ac.uk Thu Aug 22 14:...,easy_ham,0
...,...,...,...
9344,From Professional_Career_Development_Institute...,spam,1
9345,From tba@insiq.us Wed Dec 4 11:46:34 2002\nR...,spam,1
9346,Return-Path: <raye@yahoo.lv>\nReceived: from u...,spam,1
9347,From cweqx@dialix.oz.au Tue Aug 6 11:03:54 2...,spam,1


In [43]:
from sklearn.base import BaseEstimator, TransformerMixin
import email
from email_reply_parser import EmailReplyParser

# column index
content_ix = 0

class MyEmailParser(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, check_html=True): # no *args or **kargs
        self.strip_headers = strip_headers
        self.check_html = check_html
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        X_og = X.copy()
        
        if self.strip_headers:
            new_contents = []
            for email_content in X_og[:, content_ix]:
                new = ''
                message = email.message_from_string(email_content)
                if message.is_multipart():
                    for payload in message.get_payload():
                        if payload.get_content_type() == "text/plain":
                            new += payload.get_payload()     
                else:
                    new = message.get_payload()
                new_contents += [new]
            X[:, content_ix] = np.array(new_contents)
        
        if self.check_html:
            have_html = np.empty((X.shape[0], 1))
            for email_content in X_og[:, content_ix]:
                has_html = False
                if message.is_multipart():
                    for payload in message.get_payload():
                        if payload.get_content_type() == "text/html":
                            has_html = True
                else:
                    has_html = message.get_content_type() == "text/html"
                np.append(have_html, has_html)
                np.c_[X, have_html.astype(int)]
            
        return X

email_parser = MyEmailParser()
parsed_emails = email_parser.transform(emails[emails.Content.str.contains('text/html')].values[0:3, :])

In [26]:
new_contents = []
for email_content in emails[emails.Content.str.contains('text/html')].values[0:3, 0]:
    new = ''
    message = email.message_from_string(email_content)
    if message.is_multipart():
        for payload in message.get_payload():
            print(payload.get_content_type() == "text/plain")
            new += payload.get_payload()
    else:
        new = message.get_payload()
    new_contents += [new]
stripado = np.array(new_contents)

True
False
True
False


In [45]:
# print(stripado[0])

In [46]:
# print(emails[emails.Content.str.contains('text/html')].values[0, 0])

URL Parser -> detecta http e https URL_TAG -> ver beautifulsoup

HTML -> tira todas as tags

$, euro, libra, yens etc pra MONEY_TAG

numeros para NUMBER_TAG

tokenizar

stemming

tirar stopwords

selecionar só as palavras mais frequentes

checar caracteres estranho em geral, tipo chines

In [ ]:
class MyTextParser(BaseEstimator, TransformerMixin):
    def __init__(self, to_lower=True, url_subs=True, money_subs=True, number_subs=True):
        self.to_lower = to_lower
        self.url_subs = url_subs
        self.money_subs = money_subs
        self.number_subs = number_subs
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        X_og = X.copy()
        
        if to_lower:
            X[:, content_ix] = np.array([text.lower() for text in X[:, content_ix]])
            
        if url_subs:
            url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
            X[:, content_ix] = np.array([re.sub(url_regex, " url_marker ", text) for text in X[:, content_ix]])
        
        if money_subs:
            currency_regex = f"(USD)|(EUR)|(GBP)|[{''.join(chr(i) for i in range(0xffff) if unicodedata.category(chr(i)) == 'Sc')}]"
            X[:, content_ix] = np.array([re.sub(currency_regex, " currency_marker ", text) for text in X[:, content_ix]])
        
        if number_subs:
            number_regex
            X[:, content_ix] = np.array([re.sub(currency_regex, " currency_marker ", text) for text in X[:, content_ix]])
        

        return X

In [120]:
teste = "entre em http://www.banana.com ou https://www.banana.com ou http://banana.com ou www.banana.com"
re.sub("((https?\:\/\/(www\.)?)|(www\.))(\S+)(\s|$)", "URL ", teste)
re.sub(
    r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",
    "URL_MARKER",
    teste
)

'entre em URL_MARKER ou URL_MARKER ou URL_MARKER ou URL_MARKER'

In [117]:
'entre em http://www.facebook.com. Ou https://www.banana.com ou http://banana.com ou www.banana.com'

'entre em http://www.facebook.com. Ou https://www.banana.com ou http://banana.com ou www.banana.com'

In [47]:
"AAAA".lower()

'aaaa'

In [164]:
re.findall('(\d+([,.]\d+)*)', 'O valor é 123,5')

[('123,5', ',5')]

In [142]:
import unicodedata
currency_regex = f"(USD)|(EUR)|(GBP)|[{''.join(chr(i) for i in range(0xffff) if unicodedata.category(chr(i)) == 'Sc')}]"

In [143]:
currency_symbols

'(USD)|(EUR)|(GBP)|[$¢£¤¥֏؋߾߿৲৳৻૱௹฿៛₠₡₢₣₤₥₦₧₨₩₪₫€₭₮₯₰₱₲₳₴₵₶₷₸₹₺₻₼₽₾₿꠸﷼﹩＄￠￡￥￦]'

In [147]:
teste_currency = "Me da EU) 2 ou $5 ou USD24 ou £123"
re.sub(currency_symbols, "MONEY", teste_currency)

'Me da MONEY) 2 ou MONEY5 ou MONEY24 ou MONEY123'

In [169]:
!git restore

fatal: you must specify path(s) to restore


# Pipeline

contar a ocorrencia de cada palavra (binario ou frequencia?)

selecionar mais frequentes

tirar stopwords

pontuação provavelmente contar (!, ?)

tirar header do email (separar texto na primeira quebra de linha dupla (acho)) ou nao tirar?? pode avaliar assunto e quem enviou

substituir urls por URL, numeros por NUMBER, $ por MONEY talvez

stemming

tratar html </>

replies? \t |